In [2]:
!pip install pandas chromadb sentence-transformers tqdm

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import glob
import os
import re
from tqdm.notebook import tqdm
import chromadb
from sentence_transformers import SentenceTransformer

# Configure pandas and tqdm
pd.set_option('display.max_colwidth', None)
tqdm.pandas()

In [6]:
# ===================================================================
# MEMORY-EFFICIENT FAERS DATA PROCESSOR (CRASH-RESISTANT VERSION)
# ===================================================================
import pandas as pd
import glob
import os
import re
from tqdm.notebook import tqdm
import psutil
import gc
import time

# --- Configuration ---
base_directory = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/raw"
CHUNK_SIZE = 10  # Process files in chunks of 10
MEMORY_THRESHOLD = 85  # Stop if memory usage exceeds 85%

def check_memory():
    """Check current memory usage"""
    memory_percent = psutil.virtual_memory().percent
    print(f"💾 Memory usage: {memory_percent:.1f}%")
    return memory_percent

def find_files(pattern):
    """Find files with case-insensitive search"""
    search_patterns = [
        f"{base_directory}/*/ascii/{pattern}",
        f"{base_directory}/*/ASCII/{pattern}",
    ]
    
    all_files = []
    for search_path in search_patterns:
        files = glob.glob(search_path)
        all_files.extend(files)
    
    # Remove duplicates and sort by year/quarter for consistent processing
    unique_files = sorted(list(set(all_files)))
    print(f"Found {len(unique_files)} unique files for pattern {pattern}")
    return unique_files

def process_single_file(file_path):
    """Process a single file with memory management"""
    try:
        # Extract metadata
        dir_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
        match = re.match(r'(\d{4})(q\d)', dir_name)
        year, quarter = (match.groups() if match else (None, None))
        
        # Try multiple delimiters and encodings
        for delimiter in ['$', '\t', '|']:
            for encoding in ['ISO-8859-1', 'utf-8', 'cp1252']:
                try:
                    df = pd.read_csv(
                        file_path, 
                        delimiter=delimiter, 
                        encoding=encoding,
                        on_bad_lines="skip", 
                        low_memory=False,
                        dtype=str  # Read everything as string to save memory
                    )
                    
                    # Check if we got reasonable data
                    if len(df) > 0 and len(df.columns) > 1:
                        df['year'] = year
                        df['quarter'] = quarter
                        df['source_file'] = os.path.basename(file_path)
                        
                        print(f"  ✅ {os.path.basename(file_path)}: {len(df)} rows, {len(df.columns)} cols")
                        return df
                        
                except Exception as e:
                    continue
        
        print(f"  ❌ Failed to read: {os.path.basename(file_path)}")
        return None
        
    except Exception as e:
        print(f"  ❌ Error processing {file_path}: {e}")
        return None

def combine_files_chunked(file_list, output_filename, chunk_size=CHUNK_SIZE):
    """Combine files in chunks to avoid memory crashes"""
    if not file_list:
        print(f"ERROR: No files found for {output_filename}")
        return None
    
    print(f"\n🔄 Processing {len(file_list)} files in chunks of {chunk_size}")
    print(f"📁 Output: {output_filename}")
    
    # Remove existing output file if it exists
    if os.path.exists(output_filename):
        os.remove(output_filename)
        print(f"🗑️  Removed existing {output_filename}")
    
    total_rows = 0
    header_written = False
    
    # Process files in chunks
    for i in range(0, len(file_list), chunk_size):
        chunk_files = file_list[i:i+chunk_size]
        chunk_num = (i // chunk_size) + 1
        total_chunks = (len(file_list) + chunk_size - 1) // chunk_size
        
        print(f"\n📦 Processing chunk {chunk_num}/{total_chunks} ({len(chunk_files)} files)")
        
        # Check memory before processing chunk
        memory_percent = check_memory()
        if memory_percent > MEMORY_THRESHOLD:
            print(f"⚠️  Memory usage too high ({memory_percent:.1f}%). Running garbage collection...")
            gc.collect()
            time.sleep(2)
        
        chunk_dfs = []
        
        # Process each file in the chunk
        for file_path in tqdm(chunk_files, desc=f"Chunk {chunk_num}"):
            df = process_single_file(file_path)
            if df is not None:
                chunk_dfs.append(df)
        
        # Combine chunk dataframes
        if chunk_dfs:
            print(f"  🔗 Combining {len(chunk_dfs)} dataframes from chunk {chunk_num}")
            chunk_combined = pd.concat(chunk_dfs, ignore_index=True, sort=False)
            
            # Append to output file
            mode = 'w' if not header_written else 'a'
            header = not header_written
            
            chunk_combined.to_csv(output_filename, mode=mode, header=header, index=False)
            
            chunk_rows = len(chunk_combined)
            total_rows += chunk_rows
            header_written = True
            
            print(f"  ✅ Chunk {chunk_num}: {chunk_rows} rows appended (Total: {total_rows})")
            
            # Clear memory
            del chunk_combined
            del chunk_dfs
            gc.collect()
        
        print(f"  💾 Memory after chunk {chunk_num}: {psutil.virtual_memory().percent:.1f}%")
    
    if total_rows > 0:
        print(f"\n🎉 Successfully created {output_filename} with {total_rows:,} total rows")
        return pd.read_csv(output_filename, nrows=0)  # Return just headers for validation
    else:
        print(f"\n❌ No data processed for {output_filename}")
        return None

def process_data_type(data_type, patterns, resume_from=None):
    """Process a single data type with resume capability"""
    print(f"\n{'='*80}")
    print(f"🚀 PROCESSING: {data_type.upper()}")
    print(f"{'='*80}")
    
    # Check if already processed
    output_files = {
        'demo_df': 'faers_demographics_combined.csv',
        'drug_df': 'faers_drugs_combined.csv',
        'reac_df': 'faers_reactions_combined.csv',
        'indi_df': 'faers_indications_combined.csv',
        'outc_df': 'faers_outcomes_combined.csv',
        'rpsr_df': 'faers_reports_combined.csv',
        'ther_df': 'faers_therapy_combined.csv'
    }
    
    output_filename = output_files.get(data_type, f'faers_{data_type}_combined.csv')
    
    if os.path.exists(output_filename) and resume_from != data_type:
        print(f"✅ {output_filename} already exists. Skipping...")
        try:
            df = pd.read_csv(output_filename, nrows=100)  # Just check first 100 rows
            print(f"   File appears valid with {len(df.columns)} columns")
            return df
        except:
            print(f"   File appears corrupted. Will reprocess...")
    
    # Find files
    all_files = []
    for pattern in patterns:
        files = find_files(pattern)
        all_files.extend(files)
    
    unique_files = sorted(list(set(all_files)))
    
    if not unique_files:
        print(f"❌ No files found for {data_type}")
        return None
    
    print(f"📊 Found {len(unique_files)} files to process")
    
    # Process files
    return combine_files_chunked(unique_files, output_filename)

# --- MAIN EXECUTION WITH RESUME CAPABILITY ---
def main(resume_from=None):
    """
    Main function with resume capability
    resume_from: Skip to this data type (e.g., 'drug_df' to resume from drugs)
    """
    print("🚀 MEMORY-EFFICIENT FAERS PROCESSOR")
    print("="*80)
    check_memory()
    
    file_types = {
        'demo_df': ['demo*.txt', 'DEMO*.txt'],
        'drug_df': ['drug*.txt', 'DRUG*.txt'],
        'reac_df': ['reac*.txt', 'REAC*.txt'],
        'indi_df': ['indi*.txt', 'INDI*.txt'],
        'outc_df': ['outc*.txt', 'OUTC*.txt'],
        'rpsr_df': ['rpsr*.txt', 'RPSR*.txt'],
        'ther_df': ['ther*.txt', 'THER*.txt']
    }
    
    data_frames = {}
    processing_order = list(file_types.keys())
    
    # Find start point if resuming
    start_idx = 0
    if resume_from and resume_from in processing_order:
        start_idx = processing_order.index(resume_from)
        print(f"🔄 RESUMING from {resume_from}")
    
    # Process each data type
    for data_type in processing_order[start_idx:]:
        patterns = file_types[data_type]
        
        try:
            result = process_data_type(data_type, patterns, resume_from)
            if result is not None:
                data_frames[data_type] = result
            
        except KeyboardInterrupt:
            print(f"\n⚠️  Processing interrupted at {data_type}")
            print(f"To resume, run: main(resume_from='{data_type}')")
            return data_frames
            
        except Exception as e:
            print(f"\n❌ Error processing {data_type}: {e}")
            print(f"To resume, run: main(resume_from='{data_type}')")
            continue
    
    print(f"\n🎉 ALL PROCESSING COMPLETE!")
    return data_frames

# Run the main function
if __name__ == "__main__":
    # To resume from a specific point, use: main(resume_from='drug_df')
    data_frames = main()
    
    print("\n📊 FINAL SUMMARY:")
    print("="*80)
    for name, df in data_frames.items():
        if df is not None:
            # Get actual file size
            output_files = {
                'demo_df': 'faers_demographics_combined.csv',
                'drug_df': 'faers_drugs_combined.csv',
                'reac_df': 'faers_reactions_combined.csv',
                'indi_df': 'faers_indications_combined.csv',
                'outc_df': 'faers_outcomes_combined.csv',
                'rpsr_df': 'faers_reports_combined.csv',
                'ther_df': 'faers_therapy_combined.csv'
            }
            filename = output_files[name]
            if os.path.exists(filename):
                size_mb = os.path.getsize(filename) / 1024 / 1024
                print(f"✅ {name}: {filename} ({size_mb:.1f} MB)")
        else:
            print(f"❌ {name}: Not processed")

🚀 MEMORY-EFFICIENT FAERS PROCESSOR
💾 Memory usage: 62.9%

🚀 PROCESSING: DEMO_DF
Found 96 unique files for pattern demo*.txt
Found 0 unique files for pattern DEMO*.txt
📊 Found 96 files to process

🔄 Processing 96 files in chunks of 10
📁 Output: faers_demographics_combined.csv

📦 Processing chunk 1/10 (10 files)
💾 Memory usage: 62.9%


Chunk 1:   0%|          | 0/10 [00:00<?, ?it/s]

  ✅ demo13q1.txt: 223016 rows, 25 cols
  ✅ demo13q1.txt: 223016 rows, 25 cols
  ✅ demo13q2.txt: 171764 rows, 25 cols
  ✅ demo13q2.txt: 171764 rows, 25 cols
  ✅ demo13q3.txt: 185569 rows, 25 cols
  ✅ demo13q3.txt: 185569 rows, 25 cols
  ✅ demo13q4.txt: 232247 rows, 25 cols
  ✅ demo13q4.txt: 232247 rows, 25 cols
  ✅ demo14q1.txt: 260057 rows, 25 cols
  ✅ demo14q1.txt: 260057 rows, 25 cols
  🔗 Combining 10 dataframes from chunk 1
  ✅ Chunk 1: 2145306 rows appended (Total: 2145306)
  💾 Memory after chunk 1: 62.5%

📦 Processing chunk 2/10 (10 files)
💾 Memory usage: 62.5%


Chunk 2:   0%|          | 0/10 [00:00<?, ?it/s]

  ✅ demo14q2.txt: 223845 rows, 25 cols
  ✅ demo14q2.txt: 223845 rows, 25 cols
  ✅ demo14q3.txt: 211308 rows, 28 cols
  ✅ demo14q3.txt: 211308 rows, 28 cols
  ✅ demo14q4.txt: 207964 rows, 28 cols
  ✅ demo14q4.txt: 207964 rows, 28 cols
  ✅ demo15q1.txt: 317071 rows, 28 cols
  ✅ demo15q1.txt: 317071 rows, 28 cols
  ✅ demo15q2.txt: 289270 rows, 28 cols
  ✅ demo15q2.txt: 289270 rows, 28 cols
  🔗 Combining 10 dataframes from chunk 2
  ✅ Chunk 2: 2498916 rows appended (Total: 4644222)
  💾 Memory after chunk 2: 60.8%

📦 Processing chunk 3/10 (10 files)
💾 Memory usage: 60.8%


...

  ✅ THER21Q4.txt: 671045 rows, 10 cols
  ✅ THER21Q4.txt: 671045 rows, 10 cols
  ✅ THER22Q1.txt: 748899 rows, 10 cols
  ✅ THER22Q1.txt: 748899 rows, 10 cols
  ✅ THER22Q2.txt: 690828 rows, 10 cols
  ✅ THER22Q2.txt: 690828 rows, 10 cols
  ✅ THER22Q3.txt: 717902 rows, 10 cols
  ✅ THER22Q3.txt: 717902 rows, 10 cols
  ✅ THER22Q4.txt: 726767 rows, 10 cols
  ✅ THER22Q4.txt: 726767 rows, 10 cols
  🔗 Combining 10 dataframes from chunk 8
  ✅ Chunk 8: 7110882 rows appended (Total: 42941548)
  💾 Memory after chunk 8: 53.8%

📦 Processing chunk 9/10 (10 files)
💾 Memory usage: 53.8%


Chunk 9:   0%|          | 0/10 [00:00<?, ?it/s]

  ✅ THER23Q1.txt: 683408 rows, 10 cols
  ✅ THER23Q1.txt: 683408 rows, 10 cols
  ✅ THER23Q2.txt: 678128 rows, 10 cols
  ✅ THER23Q2.txt: 678128 rows, 10 cols
  ✅ THER23Q3.txt: 593027 rows, 10 cols
  ✅ THER23Q3.txt: 593027 rows, 10 cols
  ✅ THER23Q4.txt: 633087 rows, 10 cols
  ✅ THER23Q4.txt: 633087 rows, 10 cols
  ✅ THER24Q1.txt: 594449 rows, 10 cols
  ✅ THER24Q1.txt: 594449 rows, 10 cols
  🔗 Combining 10 dataframes from chunk 9
  ✅ Chunk 9: 6364198 rows appended (Total: 49305746)
  💾 Memory after chunk 9: 56.3%

📦 Processing chunk 10/10 (6 files)
💾 Memory usage: 56.3%


Chunk 10:   0%|          | 0/6 [00:00<?, ?it/s]

  ✅ THER24Q2.txt: 539334 rows, 10 cols
  ✅ THER24Q2.txt: 539334 rows, 10 cols
  ✅ THER24Q3.txt: 532854 rows, 10 cols
  ✅ THER24Q3.txt: 532854 rows, 10 cols
  ✅ THER24Q4.txt: 561889 rows, 10 cols
  ✅ THER24Q4.txt: 561889 rows, 10 cols
  🔗 Combining 6 dataframes from chunk 10
  ✅ Chunk 10: 3268154 rows appended (Total: 52573900)
  💾 Memory after chunk 10: 57.8%

🎉 Successfully created faers_therapy_combined.csv with 52,573,900 total rows

🎉 ALL PROCESSING COMPLETE!

📊 FINAL SUMMARY:
✅ demo_df: faers_demographics_combined.csv (5507.9 MB)
✅ drug_df: faers_drugs_combined.csv (16206.8 MB)
✅ reac_df: faers_reactions_combined.csv (6326.7 MB)
✅ indi_df: faers_indications_combined.csv (6124.1 MB)
✅ outc_df: faers_outcomes_combined.csv (1076.3 MB)
✅ rpsr_df: faers_reports_combined.csv (67.9 MB)
✅ ther_df: faers_therapy_combined.csv (2837.6 MB)


In [7]:
# ===================================================================
# MEMORY-EFFICIENT FAERS DATA PROCESSOR (2020-2024 ONLY)
# ===================================================================
import pandas as pd
import glob
import os
import re
from tqdm.notebook import tqdm
import psutil
import gc
import time

# --- Configuration ---
base_directory = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/raw"
CHUNK_SIZE = 10  # Process files in chunks of 10
MEMORY_THRESHOLD = 85  # Stop if memory usage exceeds 85%
TARGET_YEARS = [2020, 2021, 2022, 2023, 2024]  # 🆕 ONLY THESE YEARS

def check_memory():
    """Check current memory usage"""
    memory_percent = psutil.virtual_memory().percent
    print(f"💾 Memory usage: {memory_percent:.1f}%")
    return memory_percent

def filter_files_by_year(file_list, target_years=TARGET_YEARS):
    """Filter files to only include target years"""
    filtered_files = []
    
    for file_path in file_list:
        dir_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
        match = re.match(r'(\d{4})(q\d)', dir_name)
        
        if match:
            year = int(match.group(1))
            if year in target_years:
                filtered_files.append(file_path)
                print(f"  ✅ Including: {os.path.basename(file_path)} ({year})")
            else:
                print(f"  ⏭️  Skipping: {os.path.basename(file_path)} ({year}) - outside target range")
        else:
            print(f"  ❓ Could not extract year from: {file_path}")
    
    print(f"\n📊 Filtered {len(file_list)} files → {len(filtered_files)} files ({min(target_years)}-{max(target_years)})")
    return filtered_files

def find_files(pattern):
    """Find files with case-insensitive search and year filtering"""
    search_patterns = [
        f"{base_directory}/*/ascii/{pattern}",
        f"{base_directory}/*/ASCII/{pattern}",
    ]
    
    all_files = []
    for search_path in search_patterns:
        files = glob.glob(search_path)
        all_files.extend(files)
    
    # Remove duplicates
    unique_files = sorted(list(set(all_files)))
    print(f"Found {len(unique_files)} total files for pattern {pattern}")
    
    # 🆕 FILTER BY YEAR BEFORE PROCESSING
    filtered_files = filter_files_by_year(unique_files, TARGET_YEARS)
    
    return filtered_files

def process_single_file(file_path):
    """Process a single file with memory management"""
    try:
        # Extract metadata
        dir_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
        match = re.match(r'(\d{4})(q\d)', dir_name)
        year, quarter = (match.groups() if match else (None, None))
        
        # 🆕 DOUBLE-CHECK YEAR (safety net)
        if year and int(year) not in TARGET_YEARS:
            print(f"  ⏭️  Skipping {os.path.basename(file_path)} - year {year} not in target range")
            return None
        
        # Try multiple delimiters and encodings
        for delimiter in ['$', '\t', '|']:
            for encoding in ['ISO-8859-1', 'utf-8', 'cp1252']:
                try:
                    df = pd.read_csv(
                        file_path, 
                        delimiter=delimiter, 
                        encoding=encoding,
                        on_bad_lines="skip", 
                        low_memory=False,
                        dtype=str  # Read everything as string to save memory
                    )
                    
                    # Check if we got reasonable data
                    if len(df) > 0 and len(df.columns) > 1:
                        df['year'] = year
                        df['quarter'] = quarter
                        df['source_file'] = os.path.basename(file_path)
                        
                        print(f"  ✅ {os.path.basename(file_path)}: {len(df)} rows, {len(df.columns)} cols ({year})")
                        return df
                        
                except Exception as e:
                    continue
        
        print(f"  ❌ Failed to read: {os.path.basename(file_path)}")
        return None
        
    except Exception as e:
        print(f"  ❌ Error processing {file_path}: {e}")
        return None

def combine_files_chunked(file_list, output_filename, chunk_size=CHUNK_SIZE):
    """Combine files in chunks to avoid memory crashes"""
    if not file_list:
        print(f"ERROR: No files found for {output_filename}")
        return None
    
    print(f"\n🔄 Processing {len(file_list)} files in chunks of {chunk_size}")
    print(f"📁 Output: {output_filename}")
    
    # 🆕 ADD YEAR RANGE TO FILENAME
    year_range = f"_{min(TARGET_YEARS)}_{max(TARGET_YEARS)}"
    output_filename = output_filename.replace('.csv', f'{year_range}.csv')
    
    # Remove existing output file if it exists
    if os.path.exists(output_filename):
        os.remove(output_filename)
        print(f"🗑️  Removed existing {output_filename}")
    
    total_rows = 0
    header_written = False
    
    # Process files in chunks
    for i in range(0, len(file_list), chunk_size):
        chunk_files = file_list[i:i+chunk_size]
        chunk_num = (i // chunk_size) + 1
        total_chunks = (len(file_list) + chunk_size - 1) // chunk_size
        
        print(f"\n📦 Processing chunk {chunk_num}/{total_chunks} ({len(chunk_files)} files)")
        
        # Check memory before processing chunk
        memory_percent = check_memory()
        if memory_percent > MEMORY_THRESHOLD:
            print(f"⚠️  Memory usage too high ({memory_percent:.1f}%). Running garbage collection...")
            gc.collect()
            time.sleep(2)
        
        chunk_dfs = []
        
        # Process each file in the chunk
        for file_path in tqdm(chunk_files, desc=f"Chunk {chunk_num}"):
            df = process_single_file(file_path)
            if df is not None:
                chunk_dfs.append(df)
        
        # Combine chunk dataframes
        if chunk_dfs:
            print(f"  🔗 Combining {len(chunk_dfs)} dataframes from chunk {chunk_num}")
            chunk_combined = pd.concat(chunk_dfs, ignore_index=True, sort=False)
            
            # Append to output file
            mode = 'w' if not header_written else 'a'
            header = not header_written
            
            chunk_combined.to_csv(output_filename, mode=mode, header=header, index=False)
            
            chunk_rows = len(chunk_combined)
            total_rows += chunk_rows
            header_written = True
            
            print(f"  ✅ Chunk {chunk_num}: {chunk_rows} rows appended (Total: {total_rows})")
            
            # Clear memory
            del chunk_combined
            del chunk_dfs
            gc.collect()
        
        print(f"  💾 Memory after chunk {chunk_num}: {psutil.virtual_memory().percent:.1f}%")
    
    if total_rows > 0:
        print(f"\n🎉 Successfully created {output_filename} with {total_rows:,} total rows")
        return pd.read_csv(output_filename, nrows=0)  # Return just headers for validation
    else:
        print(f"\n❌ No data processed for {output_filename}")
        return None

def process_data_type(data_type, patterns, resume_from=None):
    """Process a single data type with resume capability"""
    print(f"\n{'='*80}")
    print(f"🚀 PROCESSING: {data_type.upper()} ({min(TARGET_YEARS)}-{max(TARGET_YEARS)})")
    print(f"{'='*80}")
    
    # 🆕 UPDATE OUTPUT FILENAMES WITH YEAR RANGE
    year_suffix = f"_{min(TARGET_YEARS)}_{max(TARGET_YEARS)}"
    output_files = {
        'demo_df': f'faers_demographics_combined{year_suffix}.csv',
        'drug_df': f'faers_drugs_combined{year_suffix}.csv',
        'reac_df': f'faers_reactions_combined{year_suffix}.csv',
        'indi_df': f'faers_indications_combined{year_suffix}.csv',
        'outc_df': f'faers_outcomes_combined{year_suffix}.csv',
        'rpsr_df': f'faers_reports_combined{year_suffix}.csv',
        'ther_df': f'faers_therapy_combined{year_suffix}.csv'
    }
    
    output_filename = output_files.get(data_type, f'faers_{data_type}_combined{year_suffix}.csv')
    
    if os.path.exists(output_filename) and resume_from != data_type:
        print(f"✅ {output_filename} already exists. Skipping...")
        try:
            df = pd.read_csv(output_filename, nrows=100)  # Just check first 100 rows
            print(f"   File appears valid with {len(df.columns)} columns")
            return df
        except:
            print(f"   File appears corrupted. Will reprocess...")
    
    # Find files (already filtered by year in find_files function)
    all_files = []
    for pattern in patterns:
        files = find_files(pattern)
        all_files.extend(files)
    
    unique_files = sorted(list(set(all_files)))
    
    if not unique_files:
        print(f"❌ No files found for {data_type} in years {TARGET_YEARS}")
        return None
    
    print(f"📊 Found {len(unique_files)} files to process for {TARGET_YEARS}")
    
    # Process files
    return combine_files_chunked(unique_files, output_filename.replace(year_suffix, ''))

# --- MAIN EXECUTION WITH YEAR FILTERING ---
def main(resume_from=None, target_years=None):
    """
    Main function with resume capability and year filtering
    resume_from: Skip to this data type (e.g., 'drug_df' to resume from drugs)
    target_years: List of years to process (default: 2020-2024)
    """
    global TARGET_YEARS
    if target_years:
        TARGET_YEARS = target_years
    
    print("🚀 MEMORY-EFFICIENT FAERS PROCESSOR")
    print(f"📅 TARGET YEARS: {TARGET_YEARS}")
    print("="*80)
    check_memory()
    
    file_types = {
        'demo_df': ['demo*.txt', 'DEMO*.txt'],
        'drug_df': ['drug*.txt', 'DRUG*.txt'],
        'reac_df': ['reac*.txt', 'REAC*.txt'],
        'indi_df': ['indi*.txt', 'INDI*.txt'],
        'outc_df': ['outc*.txt', 'OUTC*.txt'],
        'rpsr_df': ['rpsr*.txt', 'RPSR*.txt'],
        'ther_df': ['ther*.txt', 'THER*.txt']
    }
    
    data_frames = {}
    processing_order = list(file_types.keys())
    
    # Find start point if resuming
    start_idx = 0
    if resume_from and resume_from in processing_order:
        start_idx = processing_order.index(resume_from)
        print(f"🔄 RESUMING from {resume_from}")
    
    # Process each data type
    for data_type in processing_order[start_idx:]:
        patterns = file_types[data_type]
        
        try:
            result = process_data_type(data_type, patterns, resume_from)
            if result is not None:
                data_frames[data_type] = result
            
        except KeyboardInterrupt:
            print(f"\n⚠️  Processing interrupted at {data_type}")
            print(f"To resume, run: main(resume_from='{data_type}')")
            return data_frames
            
        except Exception as e:
            print(f"\n❌ Error processing {data_type}: {e}")
            print(f"To resume, run: main(resume_from='{data_type}')")
            continue
    
    print(f"\n🎉 ALL PROCESSING COMPLETE FOR {TARGET_YEARS}!")
    return data_frames

# Run the main function
if __name__ == "__main__":
    # 🆕 EXAMPLES OF HOW TO RUN:
    
    # Default: 2020-2024 only
    data_frames = main()
    
    # Custom year range:
    # data_frames = main(target_years=[2022, 2023, 2024])
    
    # Resume from specific point:
    # data_frames = main(resume_from='drug_df')
    
    # Both custom years and resume:
    # data_frames = main(resume_from='drug_df', target_years=[2020, 2021])
    
    print("\n📊 FINAL SUMMARY:")
    print("="*80)
    for name, df in data_frames.items():
        if df is not None:
            year_suffix = f"_{min(TARGET_YEARS)}_{max(TARGET_YEARS)}"
            output_files = {
                'demo_df': f'faers_demographics_combined{year_suffix}.csv',
                'drug_df': f'faers_drugs_combined{year_suffix}.csv',
                'reac_df': f'faers_reactions_combined{year_suffix}.csv',
                'indi_df': f'faers_indications_combined{year_suffix}.csv',
                'outc_df': f'faers_outcomes_combined{year_suffix}.csv',
                'rpsr_df': f'faers_reports_combined{year_suffix}.csv',
                'ther_df': f'faers_therapy_combined{year_suffix}.csv'
            }
            filename = output_files[name]
            if os.path.exists(filename):
                size_mb = os.path.getsize(filename) / 1024 / 1024
                print(f"✅ {name}: {filename} ({size_mb:.1f} MB)")
        else:
            print(f"❌ {name}: Not processed")

🚀 MEMORY-EFFICIENT FAERS PROCESSOR
📅 TARGET YEARS: [2020, 2021, 2022, 2023, 2024]
💾 Memory usage: 85.9%

🚀 PROCESSING: DEMO_DF (2020-2024)
Found 96 total files for pattern demo*.txt
  ⏭️  Skipping: demo13q1.txt (2013) - outside target range
  ⏭️  Skipping: demo13q1.txt (2013) - outside target range
  ⏭️  Skipping: demo13q2.txt (2013) - outside target range
  ⏭️  Skipping: demo13q2.txt (2013) - outside target range
  ⏭️  Skipping: demo13q3.txt (2013) - outside target range
  ⏭️  Skipping: demo13q3.txt (2013) - outside target range
  ⏭️  Skipping: demo13q4.txt (2013) - outside target range
  ⏭️  Skipping: demo13q4.txt (2013) - outside target range
  ⏭️  Skipping: demo14q1.txt (2014) - outside target range
  ⏭️  Skipping: demo14q1.txt (2014) - outside target range
  ⏭️  Skipping: demo14q2.txt (2014) - outside target range
  ⏭️  Skipping: demo14q2.txt (2014) - outside target range
  ⏭️  Skipping: demo14q3.txt (2014) - outside target range
  ⏭️  Skipping: demo14q3.txt (2014) - outside targe

Chunk 1:   0%|          | 0/10 [00:00<?, ?it/s]

  ✅ demo20q1.txt: 460327 rows, 28 cols (2020)
  ✅ demo20q1.txt: 460327 rows, 28 cols (2020)
  ✅ demo20q2.txt: 429227 rows, 28 cols (2020)
  ✅ demo20q2.txt: 429227 rows, 28 cols (2020)
  ✅ demo20q3.txt: 431667 rows, 28 cols (2020)
  ✅ demo20q3.txt: 431667 rows, 28 cols (2020)
  ✅ demo20q4.txt: 436148 rows, 28 cols (2020)
  ✅ demo20q4.txt: 436148 rows, 28 cols (2020)
  ✅ demo21q1.txt: 463741 rows, 28 cols (2021)
  ✅ demo21q1.txt: 463741 rows, 28 cols (2021)
  🔗 Combining 10 dataframes from chunk 1
  ✅ Chunk 1: 4442220 rows appended (Total: 4442220)
  💾 Memory after chunk 1: 61.4%

📦 Processing chunk 2/4 (10 files)
💾 Memory usage: 61.4%


Chunk 2:   0%|          | 0/10 [00:00<?, ?it/s]

  ✅ demo21q2.txt: 479945 rows, 28 cols (2021)
  ✅ demo21q2.txt: 479945 rows, 28 cols (2021)
  ✅ demo21q3.txt: 504160 rows, 28 cols (2021)
  ✅ demo21q3.txt: 504160 rows, 28 cols (2021)
  ✅ demo21q4.txt: 412542 rows, 28 cols (2021)
  ✅ demo21q4.txt: 412542 rows, 28 cols (2021)
  ✅ demo22q1.txt: 461623 rows, 28 cols (2022)
  ✅ demo22q1.txt: 461623 rows, 28 cols (2022)
  ✅ demo22q2.txt: 435618 rows, 28 cols (2022)
  ✅ demo22q2.txt: 435618 rows, 28 cols (2022)
  🔗 Combining 10 dataframes from chunk 2
  ✅ Chunk 2: 4587776 rows appended (Total: 9029996)
  💾 Memory after chunk 2: 55.6%

📦 Processing chunk 3/4 (10 files)
💾 Memory usage: 55.6%


...

  ✅ THER21Q2.txt: 789968 rows, 10 cols (2021)
  ✅ THER21Q2.txt: 789968 rows, 10 cols (2021)
  ✅ THER21Q3.txt: 848661 rows, 10 cols (2021)
  ✅ THER21Q3.txt: 848661 rows, 10 cols (2021)
  ✅ THER21Q4.txt: 671045 rows, 10 cols (2021)
  ✅ THER21Q4.txt: 671045 rows, 10 cols (2021)
  ✅ THER22Q1.txt: 748899 rows, 10 cols (2022)
  ✅ THER22Q1.txt: 748899 rows, 10 cols (2022)
  ✅ THER22Q2.txt: 690828 rows, 10 cols (2022)
  ✅ THER22Q2.txt: 690828 rows, 10 cols (2022)
  🔗 Combining 10 dataframes from chunk 2
  ✅ Chunk 2: 7498802 rows appended (Total: 14457808)
  💾 Memory after chunk 2: 58.7%

📦 Processing chunk 3/4 (10 files)
💾 Memory usage: 58.7%


Chunk 3:   0%|          | 0/10 [00:00<?, ?it/s]

  ✅ THER22Q3.txt: 717902 rows, 10 cols (2022)
  ✅ THER22Q3.txt: 717902 rows, 10 cols (2022)
  ✅ THER22Q4.txt: 726767 rows, 10 cols (2022)
  ✅ THER22Q4.txt: 726767 rows, 10 cols (2022)
  ✅ THER23Q1.txt: 683408 rows, 10 cols (2023)
  ✅ THER23Q1.txt: 683408 rows, 10 cols (2023)
  ✅ THER23Q2.txt: 678128 rows, 10 cols (2023)
  ✅ THER23Q2.txt: 678128 rows, 10 cols (2023)
  ✅ THER23Q3.txt: 593027 rows, 10 cols (2023)
  ✅ THER23Q3.txt: 593027 rows, 10 cols (2023)
  🔗 Combining 10 dataframes from chunk 3
  ✅ Chunk 3: 6798464 rows appended (Total: 21256272)
  💾 Memory after chunk 3: 66.9%

📦 Processing chunk 4/4 (10 files)
💾 Memory usage: 66.9%


Chunk 4:   0%|          | 0/10 [00:00<?, ?it/s]

  ✅ THER23Q4.txt: 633087 rows, 10 cols (2023)
  ✅ THER23Q4.txt: 633087 rows, 10 cols (2023)
  ✅ THER24Q1.txt: 594449 rows, 10 cols (2024)
  ✅ THER24Q1.txt: 594449 rows, 10 cols (2024)
  ✅ THER24Q2.txt: 539334 rows, 10 cols (2024)
  ✅ THER24Q2.txt: 539334 rows, 10 cols (2024)
  ✅ THER24Q3.txt: 532854 rows, 10 cols (2024)
  ✅ THER24Q3.txt: 532854 rows, 10 cols (2024)
  ✅ THER24Q4.txt: 561889 rows, 10 cols (2024)
  ✅ THER24Q4.txt: 561889 rows, 10 cols (2024)
  🔗 Combining 10 dataframes from chunk 4
  ✅ Chunk 4: 5723226 rows appended (Total: 26979498)
  💾 Memory after chunk 4: 58.8%

🎉 Successfully created faers_therapy_combined_2020_2024.csv with 26,979,498 total rows

🎉 ALL PROCESSING COMPLETE FOR [2020, 2021, 2022, 2023, 2024]!

📊 FINAL SUMMARY:
✅ demo_df: faers_demographics_combined_2020_2024.csv (2752.1 MB)
✅ drug_df: faers_drugs_combined_2020_2024.csv (8985.0 MB)
✅ reac_df: faers_reactions_combined_2020_2024.csv (3301.0 MB)
✅ indi_df: faers_indications_combined_2020_2024.csv (3314.1 